In [59]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo
import time

In [60]:
# Save output as text file to simplify the process
textfile = 'textfile.txt'

def savetofile(filename,contents):
    file = open(filename,"w",encoding="utf-8")
    file.write(contents)
    file.close()
    
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/Tyler/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [61]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

browser.visit(url)
time.sleep(3)
html = browser.html
soup = bs(html, 'html.parser')
savetofile(textfile,soup.prettify())

slides = soup.find_all('li', class_='slide')

savetofile(textfile,slides[0].prettify())

In [62]:
# Collecting the latest news title
content_title = slides[0].find('div', class_ = 'content_title')
news_title = content_title.text.strip()
news_title

'Independent Review Indicates NASA Prepared for Mars Sample Return Campaign'

In [63]:
# Collecting the latest news paragraph text
article_teaser_body = slides[0].find('div', class_ = 'article_teaser_body')
news_p = article_teaser_body.text.strip()
news_p

'NASA released an independent review report Tuesday indicating the agency is well positioned for its Mars Sample Return campaign to bring pristine samples from Mars to Earth for scientific study.'

In [64]:
# Create "Base URL" and full URL
base_url = 'https://www.jpl.nasa.gov'
url = base_url + '/spaceimages/?search=&category=Mars'

browser.visit(url)
time.sleep(3)
html = browser.html
soup = bs(html, 'html.parser')
savetofile(textfile,soup.prettify())

In [65]:
# Featured image URL
featured_image_url = base_url + soup.find('a',class_='button fancybox')['data-fancybox-href']
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18907_ip.jpg'

In [66]:
# URL of page to be scraped
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [69]:
# Use pandas to read the html table data on the page into a list of dictionaries
table = pd.read_html(url)

# Read the first dictionary in the list into a pandas dataframe and name columns
df = table[0]
df.columns = ['Mars Facts', 'Value']

df.set_index('Mars Facts', inplace=True)
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Mars Facts</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

In [ ]:
# URL of page to be scraped
base_url = 'https://astrogeology.usgs.gov'
url = base_url + '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url)

html = browser.html
soup = bs(html, 'html.parser')
savetofile(textfile,soup.prettify())

items = soup.find_all('div', class_='item')
savetofile(textfile,items[0].prettify())

In [53]:
urls = []
titles = []
for item in items:
    urls.append(base_url + item.find('a')['href'])
    titles.append(item.find('h3').text.strip())
print(urls)
titles

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']


['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [54]:
browser.visit(urls[0])
html = browser.html
soup = bs(html, 'html.parser')
savetofile(textfile,soup.prettify())
first_url = base_url+soup.find('img',class_='wide-image')['src']
first_url

'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'

In [55]:
img_urls = []
for first_url in urls:
    browser.visit(first_url)
    html = browser.html
    soup = bs(html, 'html.parser')
    savetofile(textfile,soup.prettify())
    oneurl = base_url+soup.find('img',class_='wide-image')['src']
    img_urls.append(first_url)
    
img_urls

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [56]:
hemisphere_image_urls = []

for i in range(len(titles)):
    hemisphere_image_urls.append({'title':titles[i],'img_url':img_urls[i]})

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'}]

In [57]:
for i in range(len(hemisphere_image_urls)):
    print(hemisphere_image_urls[i]['title'])
    print(hemisphere_image_urls[i]['img_url'] + '\n')

Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced

Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced

Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced

Valles Marineris Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced

